In [1]:
import pandas as pd
import sys
import os
sys.path.append("/Users/marc/Desktop/PostDoc/Mutation/signature5/notebooks")
import tools

col_order = ['Chr', 'Pos', 'Ref', 'Alt', 'Sample', 'Tissue_CellType', 'Compartment',
             'Individual', 'Study', 'Context', 'GeneralType', 'Type']
col_meta = ["Study", "Sample", "Individual", "Age", "Compartment", "Status", "Tissue_CellType"]

In [2]:
def clean_indv(s):
    return "".join([c for c in s if c.isdigit() or c.isupper()])

Read data

In [3]:
meta = pd.read_csv("Lung_organoids_telomeres_with_contamination_20190408.txt",sep="\t")

In [4]:
meta["Individual"] = meta.Sample.apply(clean_indv)

In [5]:
dat_l = []
for indv in meta.Individual.unique():
    df = pd.read_excel("Bronchial epithelium subs.xlsx", sheet_name=indv)
    df["Chr"] = "chr" + df.Chrom.astype(str)
    df["Cell"] = df.Sample
    df["Study"] = "Yoshida2020"
    df = df.rename(columns={"Type": "SType"})
    dat_l.append(df)
dat = pd.concat(dat_l)

In [43]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg19.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [48]:
dat = tools.keep_snps(dat).reset_index(drop="True")
dat = dat[~dat["Chr"].isin(["chrX", "chrY"])]
dat["Individual"] = [clean_indv(i) for i in dat.Sample]
dat["Compartment"] = "WGS"

In [ ]:
#dat = pd.read_csv("Yoshida2020.pre.csv.gz")
#dat["Sample"] = dat["Cell"]
#dat["Individual"] = [clean_indv(i) for i in dat.Sample]
#dat["Compartment"] = "WGS"

#cosmic = tools.COSMIC()
#dat = tools.annotate_context(dat, "../fasta/hg19.fa")
#dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
#dat['Type'] = cosmic.fold(dat['GeneralType'])

In [46]:
dat["chrom_id"] = dat["Chr"] + dat["Pos"].astype(str) + dat["Alt"]
dat.groupby("chrom_id").size().sort_values()

chrom_id
chr1010000269G     1
chr4134024882T     1
chr4134024417T     1
chr413402375T      1
chr4134023287T     1
                  ..
chr627309171A     47
chr928576784A     48
chr1922102655A    48
chr591245852T     50
chr1210546912A    50
Length: 1598628, dtype: int64

In [49]:
dat["Tissue_CellType"] = "Lung"
dat[col_order].to_csv("Yoshida2020.csv.gz", compression="gzip", index=False)

Metafile

In [7]:
meta["Individual"] = [clean_indv(s) for s in meta.Sample]
meta["Compartment"] = "WGS"

In [8]:
meta["Tissue_CellType"] = "Lung"
meta["Status"] = meta.Smoking
meta["Study"] = "Yoshida2020"

In [3]:
!gunzip -c Yoshida2020.tab.gz | rev | cut -f2 | rev | sort | uniq -c 

 219 ex-smoker
 256 non-smoker
 157 smoker
   1 Smoking


In [12]:
meta[col_meta + ["Smoking", "PackYear"]].to_csv("Yoshida2020.tab.gz",sep="\t")